<a href="https://colab.research.google.com/github/minhhieu9800/MLE501/blob/main/Assignment_MLE501.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Giới thiệu

Họ tên: Nguyễn Minh Hiếu </br>
Lớp: MSE#11HCM </br>
Mã học viên: 22MSE23062 </br>
Email: hieu22mse23062@fsb.edu.vn

# Assignment

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from tabulate import tabulate

from keras.models import Sequential
from keras.layers import Dense
from keras.models import load_model
from keras.optimizers import Adam, RMSprop
from sklearn.model_selection import train_test_split

Load dữ liệu

In [2]:
url = 'https://minhhieu9800.github.io/MLE501/diabetes.csv'
df = pd.read_csv(url)

In [3]:
X = df.drop(['Outcome'], axis=1)
y = df['Outcome']

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2)

Xây dựng model

In [4]:
model = Sequential()
model.add(Dense(8, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.summary()

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, batch_size=8, validation_data=(X_val, y_val))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 72        
                                                                 
 dense_1 (Dense)             (None, 8)                 72        
                                                                 
 dense_2 (Dense)             (None, 1)                 9         
                                                                 
Total params: 153
Trainable params: 153
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
62/62 [==============================] - 2s 16ms/step - loss: 2.6303 - accuracy: 0.5580 - val_loss: 1.3686 - val_accuracy: 0.6098
Epoch 2/100
62/62 [==============================] - 1s 9ms/step - loss: 1.2791 - accuracy: 0.5886 - val_loss: 1.2062 - val_accuracy: 0.5203
Epoch 3/100
62/62 [====================

In [5]:
model.save("model.h5")

In [6]:
model = load_model("model.h5")

In [7]:
loss, accuracy = model.evaluate(X_test, y_test)
print("Loss: ", loss)
print("Accuracy: ", accuracy)

5/5 [==============================] - 0s 4ms/step - loss: 0.6241 - accuracy: 0.6753
Loss:  0.6241394877433777
Accuracy:  0.6753246784210205


## Sử dụng Hyperparameter Tuning (Keras Tuner)

In [8]:
!pip install keras-tuner
import keras_tuner as kt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.3/167.3 KB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 30.2 MB/s eta 0:00:00


In [9]:
# Tạm hàm build Hyper model
def build_model(hp): 
  model = keras.Sequential()

  X = hp.Int('units_input', min_value=8, max_value=64, step=8)
  model.add(Dense(units=X, input_dim=8, activation='relu'))

  for i in range(hp.Int('num_layers', 1, 4)):
    Y = hp.Int('units_' + str(i), min_value=8, max_value=64, step=8)
    model.add(Dense(units=Y, activation='relu'))

  model.add(Dense(1, activation='sigmoid'))

  lr = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
  model.compile(
        optimizer=Adam(learning_rate=lr),
        loss='binary_crossentropy',
        metrics=['accuracy'])
  
  return model

In [10]:
# Khởi tạo Tuner
tuner = kt.Hyperband (build_model, objective="val_accuracy", directory="tuner_dir_1", project_name="MLE501_Tuner_1")

In [11]:
tuner.search(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

Trial 254 Complete [00h 00m 15s]
val_accuracy: 0.642276406288147

Best val_accuracy So Far: 0.8455284833908081
Total elapsed time: 00h 15m 22s


In [12]:
# lấy best model
best_model = tuner.get_best_models(num_models=1)[0]

best_model.summary()

best_model.fit(X_train, y_train, epochs=100, batch_size=8, validation_data=(X_val, y_val))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 56)                504       
                                                                 
 dense_1 (Dense)             (None, 40)                2280      
                                                                 
 dense_2 (Dense)             (None, 40)                1640      
                                                                 
 dense_3 (Dense)             (None, 64)                2624      
                                                                 
 dense_4 (Dense)             (None, 1)                 65        
                                                                 
Total params: 7,113
Trainable params: 7,113
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
62/62 [==============================] - 3

In [13]:
best_model.save("best_model_hp.h5")

In [14]:
loss, accuracy = best_model.evaluate(X_test, y_test)
print("Best model using Hyperparameter Tuning")
print("Loss: ", loss)
print("Accuracy: ", accuracy)

model = load_model("model.h5")
loss, accuracy = model.evaluate(X_test, y_test)
print("Original model")
print("Loss: ", loss)
print("Accuracy: ", accuracy)

5/5 [==============================] - 0s 3ms/step - loss: 0.6079 - accuracy: 0.7208
Best model using Hyperparameter Tuning
Loss:  0.6078561544418335
Accuracy:  0.7207792401313782
5/5 [==============================] - 0s 4ms/step - loss: 0.6241 - accuracy: 0.6753
Original model
Loss:  0.6241394877433777
Accuracy:  0.6753246784210205


### So với model ban đầu, model được cải tiến bằng Hyperparameter Tuning đã có sự cải thiện về độ chính xác. Sự thay đổi tạo ra từ việc các Hyperparameter đã được tối ưu có chọn lọc.




In [15]:
# Từ ban đầu model chỉ có 3 lớp với các thông số như sau:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 72        
                                                                 
 dense_1 (Dense)             (None, 8)                 72        
                                                                 
 dense_2 (Dense)             (None, 1)                 9         
                                                                 
Total params: 153
Trainable params: 153
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Model sau khi được tối ưu có 6 lớp với các thông số sau:
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 56)                504       
                                                                 
 dense_1 (Dense)             (None, 40)                2280      
                                                                 
 dense_2 (Dense)             (None, 40)                1640      
                                                                 
 dense_3 (Dense)             (None, 64)                2624      
                                                                 
 dense_4 (Dense)             (None, 1)                 65        
                                                                 
Total params: 7,113
Trainable params: 7,113
Non-trainable params: 0
_________________________________________________________________


## Sử dụng Regularization (L2 Regularization)

In [17]:
from keras import regularizers

In [18]:
#áp dụng  với hệ số lambda là 0.01 cho cả 2 lớp Dense đầu tiên của kiến trúc mạng neural. 

model = Sequential()
model.add(Dense(16, input_dim=8, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(8, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, batch_size=8, validation_data=(X_val, y_val))

Epoch 1/100
62/62 [==============================] - 2s 8ms/step - loss: 14.4191 - accuracy: 0.5682 - val_loss: 3.6258 - val_accuracy: 0.4634
Epoch 2/100
62/62 [==============================] - 0s 7ms/step - loss: 2.5798 - accuracy: 0.4786 - val_loss: 2.2378 - val_accuracy: 0.5122
Epoch 3/100
62/62 [==============================] - 0s 7ms/step - loss: 1.7076 - accuracy: 0.5051 - val_loss: 1.7553 - val_accuracy: 0.4634
Epoch 4/100
62/62 [==============================] - 0s 7ms/step - loss: 1.4406 - accuracy: 0.5682 - val_loss: 1.0810 - val_accuracy: 0.6423
Epoch 5/100
62/62 [==============================] - 0s 6ms/step - loss: 1.1735 - accuracy: 0.5662 - val_loss: 0.9995 - val_accuracy: 0.6992
Epoch 6/100
62/62 [==============================] - 0s 5ms/step - loss: 1.1195 - accuracy: 0.6171 - val_loss: 1.1195 - val_accuracy: 0.6179
Epoch 7/100
62/62 [==============================] - 0s 6ms/step - loss: 1.0313 - accuracy: 0.5947 - val_loss: 0.9027 - val_accuracy: 0.7073
Epoch 8/100


In [19]:
model.save("best_model_regularization.h5")

In [20]:
loss, accuracy = model.evaluate(X_test, y_test)
print("Best model using Regularization (L2 Regularization)")
print("Loss: ", loss)
print("Accuracy: ", accuracy)

original = load_model("model.h5")
loss, accuracy = original.evaluate(X_test, y_test)
print("Original model")
print("Loss: ", loss)
print("Accuracy: ", accuracy)

5/5 [==============================] - 0s 3ms/step - loss: 0.6683 - accuracy: 0.7468
Best model using Regularization (L2 Regularization)
Loss:  0.668339729309082
Accuracy:  0.7467532753944397
5/5 [==============================] - 0s 4ms/step - loss: 0.6241 - accuracy: 0.6753
Original model
Loss:  0.6241394877433777
Accuracy:  0.6753246784210205


### So với model ban đầu, model được cải tiến bằng Regularization đã có sự thay đổi về độ chính xác, cụ thể ở đây em dùng kỹ thuật L2 Regularization. Sự thay đổi tạo ra từ việc thêm một thành phần bình phương của các trọng số vào hàm mất mát. Việc này sẽ làm cho các trọng số nhỏ hơn, dẫn đến mô hình đơn giản hơn và tránh được việc bị overfitting.

## Sử dụng Optimization (Adam -> RMSprop)

In [21]:
model = Sequential()
model.add(Dense(16, input_dim=8, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(8, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer=RMSprop(learning_rate=0.001), metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, batch_size=8, validation_data=(X_val, y_val))

Epoch 1/100
62/62 [==============================] - 2s 22ms/step - loss: 2.0201 - accuracy: 0.5804 - val_loss: 1.2111 - val_accuracy: 0.6179
Epoch 2/100
62/62 [==============================] - 1s 18ms/step - loss: 1.2378 - accuracy: 0.6191 - val_loss: 1.0195 - val_accuracy: 0.6341
Epoch 3/100
62/62 [==============================] - 1s 15ms/step - loss: 1.1141 - accuracy: 0.6334 - val_loss: 0.9428 - val_accuracy: 0.6829
Epoch 4/100
62/62 [==============================] - 1s 10ms/step - loss: 1.0406 - accuracy: 0.6334 - val_loss: 0.9200 - val_accuracy: 0.6504
Epoch 5/100
62/62 [==============================] - 0s 7ms/step - loss: 0.9704 - accuracy: 0.6395 - val_loss: 0.8856 - val_accuracy: 0.6748
Epoch 6/100
62/62 [==============================] - 0s 7ms/step - loss: 0.9278 - accuracy: 0.6538 - val_loss: 0.8831 - val_accuracy: 0.6585
Epoch 7/100
62/62 [==============================] - 0s 7ms/step - loss: 0.8818 - accuracy: 0.6640 - val_loss: 0.8558 - val_accuracy: 0.6667
Epoch 8/1

In [22]:
model.save("best_model_optimization.h5")

In [23]:
loss, accuracy = model.evaluate(X_test, y_test)
print("Best model using Optimization (Adam -> RMSprop)")
print("Loss: ", loss)
print("Accuracy: ", accuracy)

original = load_model("model.h5")
loss, accuracy = original.evaluate(X_test, y_test)
print("Original model")
print("Loss: ", loss)
print("Accuracy: ", accuracy)

5/5 [==============================] - 0s 4ms/step - loss: 0.6644 - accuracy: 0.7468
Best model using Optimization (Adam -> RMSprop)
Loss:  0.6643891930580139
Accuracy:  0.7467532753944397
5/5 [==============================] - 0s 4ms/step - loss: 0.6241 - accuracy: 0.6753
Original model
Loss:  0.6241394877433777
Accuracy:  0.6753246784210205


### So với model ban đầu, model được cải tiến bằng Regularization đã có sự thay đổi về độ chính xác, cụ thể ở đây em thay đổi optimizer từ Adam sang RMSprop. Thuật toán tối ưu mô hình sẽ thay đổi cách tính toán gradient và cập nhật trọng số trong quá trình huấn luyện. Cụ thể, hai thuật toán này sử dụng hai phương pháp khác nhau để tính toán gradient và cập nhật trọng số, do đó việc thay đổi optimizer sẽ có tác động đến hiệu suất của mô hình.

## Kết hợp cả 3 kỹ thuật

In [24]:
# Tạm hàm build Hyper model
def build_best_model(hp): 
  model = keras.Sequential()

  X = hp.Int('units_input', min_value=8, max_value=64, step=8)
  model.add(Dense(units=X, input_dim=8, activation='relu', kernel_regularizer=regularizers.l2(0.01)))

  for i in range(hp.Int('num_layers', 1, 4)):
    Y = hp.Int('units_' + str(i), min_value=8, max_value=64, step=8)
    model.add(Dense(units=Y, activation='relu', kernel_regularizer=regularizers.l2(0.01)))

  model.add(Dense(1, activation='sigmoid'))

  # Tìm kiếm optimizer
  optimizer = hp.Choice('optimizer', values=['adam', 'rmsprop'])

  # Tìm kiếm learning_rate
  lr = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  if optimizer == 'adam':
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=lr), metrics=['accuracy'])
  else:
    model.compile(loss='binary_crossentropy', optimizer=RMSprop(learning_rate=lr), metrics=['accuracy'])
  
  return model

In [25]:
# Khởi tạo Tuner
tuner = kt.Hyperband (build_best_model, objective="val_accuracy", directory="tuner_dir_2", project_name="MLE501_Tuner_2")

In [26]:
tuner.search(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

Trial 254 Complete [00h 00m 22s]
val_accuracy: 0.8292682766914368

Best val_accuracy So Far: 0.869918704032898
Total elapsed time: 00h 16m 54s


In [27]:
# Lấy ra các tham số
best_hps = tuner.get_best_hyperparameters()[0]

print("units_input: ", best_hps.get('units_input'))
print("num_layers: ", best_hps.get('num_layers'))
print("optimizer: ", best_hps.get('optimizer'))
print("learning_rate: ", best_hps.get('learning_rate'))

units_input:  8
num_layers:  3
optimizer:  adam
learning_rate:  0.01


In [28]:
# lấy super model
best_model = tuner.get_best_models(num_models=1)[0]

best_model.summary()

best_model.fit(X_train, y_train, epochs=100, batch_size=8, validation_data=(X_val, y_val))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 72        
                                                                 
 dense_1 (Dense)             (None, 8)                 72        
                                                                 
 dense_2 (Dense)             (None, 32)                288       
                                                                 
 dense_3 (Dense)             (None, 8)                 264       
                                                                 
 dense_4 (Dense)             (None, 1)                 9         
                                                                 
Total params: 705
Trainable params: 705
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
62/62 [==============================] - 2s 10

In [29]:
best_model.save("best_model_super.h5")

In [30]:
modelOriginal = load_model("model.h5")
modelHp = load_model("best_model_hp.h5")
modelRe = load_model("best_model_regularization.h5")
modelOp = load_model("best_model_optimization.h5")
modelSuper = load_model("best_model_super.h5")

lossOriginal, accuracyOriginal = modelOriginal.evaluate(X_test, y_test)
lossHp, accuracyHp = modelHp.evaluate(X_test, y_test)
lossRe, accuracyRe = modelRe.evaluate(X_test, y_test)
lossOp, accuracyOp = modelOp.evaluate(X_test, y_test)
lossSuper, accuracySuper = modelSuper.evaluate(X_test, y_test)

5/5 [==============================] - 0s 4ms/step - loss: 0.5448 - accuracy: 0.7403


In [31]:
# create the DataFrame
df = pd.DataFrame({'': ['Original','Hyperparameter Tuning (Keras Tuner)','Regularization (L2 Regularization)','Optimization (Adam -> RMSprop)','Combined 3 ways'],
                   'Loss': [lossOriginal, lossHp, lossRe, lossOp, lossSuper],
                   'Accuracy': [accuracyOriginal, accuracyHp, accuracyRe, accuracyOp, accuracySuper]})

# print the table using tabulate
print(tabulate(df, headers='keys', tablefmt='pipe', floatfmt='.4f'))

|    |                                     |   Loss |   Accuracy |
|---:|:------------------------------------|-------:|-----------:|
|  0 | Original                            | 0.6241 |     0.6753 |
|  1 | Hyperparameter Tuning (Keras Tuner) | 0.6079 |     0.7208 |
|  2 | Regularization (L2 Regularization)  | 0.6683 |     0.7468 |
|  3 | Optimization (Adam -> RMSprop)      | 0.6644 |     0.7468 |
|  4 | Combined 3 ways                     | 0.5448 |     0.7403 |
